<h1> Import Libraries

In [ ]:
import numpy as np 
import pandas as pd 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import glob as gb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import matplotlib.pyplot as plt
import PIL
import cv2
import random
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D,BatchNormalization
from keras.layers import Dropout
from tensorflow.keras.metrics import Precision, Recall
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import layers, Sequential, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import MobileNetV2, VGG19, DenseNet121
from tensorflow.keras.models import Model

<h1> Loading The Data

In [ ]:
BASE_DIR = '/kaggle/input/egyptian-hieroglyphs'
NEW_SIZE = 224

In [ ]:
train_dir = os.path.join(BASE_DIR, 'train')
test_dir = os.path.join(BASE_DIR, 'test')
validation_dir = os.path.join(BASE_DIR, 'valid')

In [ ]:
def print_directory_structure(directories):
    for directory in directories:
        for dirpath, dirnames, filenames in os.walk(directory):
            print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [ ]:
print_directory_structure([train_dir, test_dir, validation_dir])

In [ ]:
def load_images_and_labels(directory, df):
    X = []
    y = []
    for i, img_name in enumerate(df['filename']):
        img_path = os.path.join(directory, img_name)
        img_class = df.loc[i, 'class']
        
        try:
            original_img = cv2.imread(img_path)
            image = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)

            xmin, ymin, xmax, ymax = df.loc[i, ['xmin', 'ymin', 'xmax', 'ymax']]

            cropped_img = image[ymin:ymax, xmin:xmax]
            resized_img = cv2.resize(cropped_img, (NEW_SIZE, NEW_SIZE))
            X.append(resized_img)
            y.append(img_class)
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
    return np.array(X), np.array(y)

In [ ]:
df_train = pd.read_csv(gb.glob(train_dir + '/*.csv')[0])
X_train, y_train = load_images_and_labels(train_dir, df_train)

In [ ]:
df_valid = pd.read_csv(gb.glob(validation_dir + '/*.csv')[0])
X_valid, y_valid = load_images_and_labels(validation_dir, df_valid)

In [ ]:
df_test = pd.read_csv(gb.glob(test_dir + '/*.csv')[0])
X_test, y_test = load_images_and_labels(test_dir, df_test)

<h1>Visualisation</h1>

In [ ]:
plt.figure(figsize=(20,20))

for i, n in enumerate(list(np.random.randint(0,len(X_train), 16))):
    plt.subplot(4, 4, i+1)
    plt.imshow(X_train[i])
    plt.title(y_train[i])

In [ ]:
plt.figure(figsize=(15, 8))
df_train['class'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Distribution of Classes')
plt.xlabel('Class')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.show()

<h1> Preprocessing</h1>

In [ ]:
X_train = X_train.reshape(X_train.shape[0], NEW_SIZE, NEW_SIZE, 3)
X_valid = X_valid.reshape(X_valid.shape[0], NEW_SIZE, NEW_SIZE, 3)
X_test = X_test.reshape(X_test.shape[0], NEW_SIZE, NEW_SIZE, 3)

In [ ]:
X_train, X_valid, X_test = X_train / 255.0, X_valid / 255.0, X_test / 255.0

In [ ]:
X_train, y_train = shuffle(X_train, y_train, random_state=42)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X_train)

In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = to_categorical(label_encoder.fit_transform(y_train))
y_valid_encoded = to_categorical(label_encoder.transform(y_valid))
y_test_encoded = to_categorical(label_encoder.transform(y_test))

In [ ]:
print('X_train shape: ', X_train.shape)
print('y_train_encoded shape: ', y_train_encoded.shape)
print('X_valid shape: ', X_valid.shape)
print('y_valid_encoded shape: ', y_valid_encoded.shape)
print('X_test shape: ', X_test.shape)
print('y_test_encoded shape: ', y_test_encoded.shape)

<h1>Modeling</h1>

In [ ]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()


In [ ]:
metrics = [
    'accuracy',
    Precision(),
    Recall(),
    F1Score()
]

<h2>Normal CNN</h2>

In [ ]:
num_classes = len(df_train['class'].unique())  # Ensure you're using the training DataFrame
input_shape = (224, 224, 3)

# Build the model
model = Sequential()
model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)

In [ ]:
model.summary()

In [ ]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train) 
class_weight_dict = dict(enumerate(class_weights))

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

callbacks = [early_stopping]

In [ ]:
history = model.fit(
    X_train, 
    y_train_encoded, 
    epochs=100, 
    validation_data=(X_valid, y_valid_encoded), 
    callbacks=callbacks
)

In [ ]:
def evaluate_model(model, train_gen, valid_gen, test_gen):
    # Evaluate the model on the training data generator
    model_eval_train = model.evaluate(train_gen)
    print('Training Data:')
    print('Loss:      ', model_eval_train[0])
    print('Accuracy:  ', model_eval_train[1])
    print('Precision: ', model_eval_train[2])
    print('Recall:    ', model_eval_train[3])
    print('F1-Score:  ', model_eval_train[4])
    
    # Evaluate the model on the validation data generator
    model_eval_valid = model.evaluate(valid_gen)
    print('\nValidation Data:')
    print('Loss:      ', model_eval_valid[0])
    print('Accuracy:  ', model_eval_valid[1])
    print('Precision: ', model_eval_valid[2])
    print('Recall:    ', model_eval_valid[3])
    print('F1-Score:  ', model_eval_valid[4])
    
    # Evaluate the model on the test data generator
    model_eval_test = model.evaluate(test_gen)
    print('\nTest Data:')
    print('Loss:      ', model_eval_test[0])
    print('Accuracy:  ', model_eval_test[1])
    print('Precision: ', model_eval_test[2])
    print('Recall:    ', model_eval_test[3])
    print('F1-Score:  ', model_eval_test[4])
    
    return (np.round(model_eval_train[0], 2), np.round(model_eval_valid[0], 2), np.round(model_eval_test[0], 2),
            np.round(model_eval_train[1], 2), np.round(model_eval_valid[1], 2), np.round(model_eval_test[1], 2),
            np.round(model_eval_train[2], 2), np.round(model_eval_valid[2], 2), np.round(model_eval_test[2], 2),
            np.round(model_eval_train[3], 2), np.round(model_eval_valid[3], 2), np.round(model_eval_test[3], 2),
            np.round(model_eval_train[4], 2), np.round(model_eval_valid[4], 2), np.round(model_eval_test[4], 2))


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train_encoded)).batch(32).prefetch(tf.data.AUTOTUNE)
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid_encoded)).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test_encoded)).batch(32).prefetch(tf.data.AUTOTUNE)

evaluate_model(model, train_dataset, valid_dataset, test_dataset)


In [ ]:
Final_report = []

In [ ]:
Final_report.append(evaluate_model(model, train_dataset, valid_dataset, test_dataset))

<h2>VGG19 Model</h2>

In [ ]:
vgg19_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in vgg19_model.layers:
    layer.trainable = False

model = models.Sequential([
    vgg19_model,
    GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics= metrics)

# Fit the model on your training data
history = model.fit(X_train, y_train_encoded, epochs=100, validation_data=(X_valid, y_valid_encoded), callbacks=callbacks)

In [ ]:
Final_report.append(evaluate_model(model, train_dataset, valid_dataset, test_dataset))

<h2>MobileNetV2 Model</h2>

In [ ]:
mobilenetv2_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in mobilenetv2_model.layers:
    layer.trainable = False

In [ ]:
model = models.Sequential([
    mobilenetv2_model,
    GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=metrics)

In [ ]:
history = model.fit(
    X_train, 
    y_train_encoded, 
    epochs=100, 
    validation_data=(X_valid, y_valid_encoded), 
    callbacks=callbacks
)

In [ ]:
Final_report.append(evaluate_model(model, train_dataset, valid_dataset, test_dataset))

<h2>DenseNet121 Model</h2>

In [ ]:
densenet121_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in densenet121_model.layers:
    layer.trainable = False


In [ ]:
model = models.Sequential([
    densenet121_model,
    GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=metrics)

In [ ]:
history = model.fit(
    X_train, 
    y_train_encoded, 
    epochs=100,
    validation_data=(X_valid, y_valid_encoded), 
    callbacks= callbacks,  # Any callbacks you wish to use
)

In [ ]:
Final_report.append(evaluate_model(model, train_dataset, valid_dataset, test_dataset))

<h1>Conclusion</h1>

In [ ]:
Model_Names = ['Normal CNN', 'VGG19', 'MobileNetV2', 'DenseNet']

In [ ]:
Model_Accuracy = pd.DataFrame(
    Final_report,
    index=Model_Names,
    columns=[
        'Train Loss', 'Validation Loss', 'Test Loss',
        'Train Accuracy', 'Validation ACC', 'Test ACC',
        'Train Precision', 'Validation Precision', 'Test Precision',
        'Train Recall', 'Validation Recall', 'Test Recall',
        'Train F1-Score', 'Validation F1-Score', 'Test F1-Score'
    ]
)

In [ ]:
Model_Accuracy

In [ ]:
plt.figure(figsize=(14, 7))
bar_width = 0.25  # Width of the bars
index = np.arange(len(Model_Accuracy.index))  # Indices for x-axis

plt.bar(index, Model_Accuracy['Train Loss'], width=bar_width, label='Train Loss')
plt.bar(index + bar_width, Model_Accuracy['Validation Loss'], width=bar_width, label='Validation Loss')
plt.bar(index + 2 * bar_width, Model_Accuracy['Test Loss'], width=bar_width, label='Test Loss')

plt.title('Loss for Different Models')
plt.xlabel('Models')
plt.ylabel('Loss')
plt.xticks(index + bar_width, Model_Accuracy.index, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))
plt.bar(index, Model_Accuracy['Train Accuracy'], width=bar_width, label='Train Accuracy')
plt.bar(index + bar_width, Model_Accuracy['Validation ACC'], width=bar_width, label='Validation Accuracy')
plt.bar(index + 2 * bar_width, Model_Accuracy['Test ACC'], width=bar_width, label='Test Accuracy')

plt.title('Accuracy for Different Models')
plt.xlabel('Models')
plt.ylabel('Accuracy')
plt.xticks(index + bar_width, Model_Accuracy.index, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))
plt.bar(index, Model_Accuracy['Train Precision'], width=bar_width, label='Train Precision')
plt.bar(index + bar_width, Model_Accuracy['Validation Precision'], width=bar_width, label='Validation Precision')
plt.bar(index + 2 * bar_width, Model_Accuracy['Test Precision'], width=bar_width, label='Test Precision')

plt.title('Precision for Different Models')
plt.xlabel('Models')
plt.ylabel('Precision')
plt.xticks(index + bar_width, Model_Accuracy.index, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))
plt.bar(index, Model_Accuracy['Train Recall'], width=bar_width, label='Train Recall')
plt.bar(index + bar_width, Model_Accuracy['Validation Recall'], width=bar_width, label='Validation Recall')
plt.bar(index + 2 * bar_width, Model_Accuracy['Test Recall'], width=bar_width, label='Test Recall')

plt.title('Recall for Different Models')
plt.xlabel('Models')
plt.ylabel('Recall')
plt.xticks(index + bar_width, Model_Accuracy.index, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))
plt.bar(index, Model_Accuracy['Train F1-Score'], width=bar_width, label='Train F1-Score')
plt.bar(index + bar_width, Model_Accuracy['Validation F1-Score'], width=bar_width, label='Validation F1-Score')
plt.bar(index + 2 * bar_width, Model_Accuracy['Test F1-Score'], width=bar_width, label='Test F1-Score')

plt.title('F1-Score for Different Models')
plt.xlabel('Models')
plt.ylabel('F1-Score')
plt.xticks(index + bar_width, Model_Accuracy.index, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
Model_Accuracy.to_csv('Report.csv', index = True)